In [15]:
import pandas as pd 
import psycopg2
import plotly.express as px
import datetime
# import os


filter_broker = ['Necton','Guide','Bradesco']
today = datetime.date.today()
df_saldo=pd.read_excel(f"G:\Trading\K11\Aluguel\Arquivos\Doar\Saldo-Dia\K11_lend_complete_{datetime.date.today().strftime('%d-%m-%Y')}.xlsx",index_col=0)
# display(df_saldo)


def get_analysis(ticker):
    db_conn = psycopg2.connect(host='PGKPTL01', dbname='db_Teste' , user='kapitalo11', password='kapitalo11')
    query = f"select * from ibotz.tbl_boletasalugueis_ibotz where str_codigo in {ticker} and str_corretora not like '%Interna%' and str_mercado not like '%Devolucao%' and dbl_quantidade<0"
    df = pd.read_sql(query,db_conn)
    df['str_corretora'] = df['str_corretora'].apply(lambda x: x.replace('FC Stone','Necton'))
    db_conn.close()
    return df



df_saldo['cod'] = df_saldo['codigo'].apply(lambda x: x[:4])
df = get_analysis(tuple(df_saldo['cod'].unique()))


df = df.groupby(['str_corretora','str_codigo']).agg({'dbl_quantidade':sum}).reset_index().set_index('str_codigo').sort_values('dbl_quantidade')


for x in df_saldo['cod'].unique():
    try:
        df.loc[x,'prop'] = df.loc[x,'dbl_quantidade']/df.loc[x,'dbl_quantidade'].sum()
        # fig_a= px.bar(df.loc[x].head(),y="prop",x="str_corretora",title=x)
        # fig_a.show()
        
    except:
        pass

df_saldo = df_saldo.merge(df,left_on='cod',right_on='str_codigo',how='left')

corretoras = pd.read_excel(r'G:\Transfer\Lista de Contrapartes Autorizadas\Contrapartes Operacionais.onshore.xlsx',sheet_name='BTC',skiprows=7)

import pandas as pd
corretoras = pd.read_excel(r'G:\Transfer\Lista de Contrapartes Autorizadas\Contrapartes Operacionais.onshore.xlsx',sheet_name='BTC',skiprows=7)
corretoras = corretoras[['Contraparte', 'Master II ', 'Master I ',
       'Tarkus Master ', 'Argus Master', 'Sigma LLC ', 'Gaia',
       'Alpha Global Master', 'Estrategias Master', 'Master V',
       'Zeta Merídia Master', 'K10 Previdência', 'K10 Master', 'Ômega Prev.',
       'Kappa Prev. Master']].fillna(0)


de_para = {
    'Contraparte':'Contraparte',
    'Master I ':'KAPITALO KAPPA MASTER FIM',
    'Master II ':'KAPITALO ZETA MASTER FIM',
    'Sigma LLC ':'KAPITALO SIGMA LLC',
    'Zeta Merídia Master':'KAPITALO ZETA MASTER FIA',
    'K10 Previdência' :'KAPITALO K10 PREV MASTER FIM',
    'K10 Master':'KAPITALO K10 MASTER FIM',
    'Ômega Prev.':'KAPITALO OMEGA PREV MASTER FIM',
    'Kappa Prev. Master':'KAPITALO KAPPA PREV MASTER FIM'
}

de_para_corretoras ={
    'Ativa Investimentos':'Ativa',
    'BGC Liquidez':'Liquidez',
    'BTG Pactual CTVM':'BTG Pactual',
    'Capital Markets':'Capital Markets',
    'Citigroup CCTVM':'Citi',
    'Credit Suisse':'CS',
    'Genial Institucional ':'Plural',
    'Guide Investimentos':'Guide',
    'Inter DTVM':'Inter',
    'INTL FCStone DTVM':'FC Stone',
    'Itaú Corretora':'Itau',
    'JP Morgan CCVM':'JP Morgan',
    'Merrill Lynch CTVM':'Bank of America',
    'Mirae Asset Wealth Management':'Mirae',
    'Morgan Stanley CTVM':'Morgan Stanley',
    'Necton Investimentos':'Necton',
    'Órama DTVM':'Orama',
    'Safra Corretora de Valores e Câmbio':'Safra',
    'Santander CCVM':'Santander',
    'Terra Investimentos DTVM':'Terra',
    'Tullett Prebon Brasil':'Tullet',
    'UBS BrasiL CCTVM':'Link',
    'XP Investimentos':'XP',
}

corretoras.columns = corretoras.columns.map(de_para)

corretoras = corretoras[corretoras.columns.dropna()]
corretoras = corretoras.set_index('Contraparte')

corretoras = corretoras.reset_index()




# corretoras

corretoras['Contraparte'] = corretoras['Contraparte'].map(de_para_corretoras).dropna()
corretoras = corretoras.dropna()


corretoras = corretoras.reset_index()

corretoras = corretoras[corretoras['Contraparte'].isin(filter_broker)]

corretoras = corretoras.set_index('Contraparte')
if 'Bradesco' in filter_broker:
    corretoras.loc['Bradesco'] = 'x'

new_df = pd.DataFrame()
for x in df_saldo['fundo'].unique():
    
    try:
        lista_corretoras = corretoras[corretoras[x]!=0][[x]].index
        aux = df_saldo.loc[(df_saldo['fundo']==x)&(df_saldo['str_corretora'].isin(lista_corretoras))].copy()
        new_df = pd.concat([new_df,aux])
    except:
        pass

for fund in new_df['fundo'].unique():
    for x in new_df['codigo'].unique():
        new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop'] = new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop']/new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop'].sum()
new_df = new_df[new_df['prop']>0.30]
for fund in new_df['fundo'].unique():
    for x in new_df['codigo'].unique():
        new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop'] = new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop']/new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop'].sum()

new_df['Doador'] = new_df.apply(lambda row: round(row['to_lend']*row['prop'],0),axis=1)

import datetime

geral = new_df[~new_df['codigo'].isin(['BOVA11','BRFS3','GOLL4','GFSA3','CMIN3','LEVE3','AMPB3','TRPL4','CVCB3','ASAI3','ROXO34'])].copy()
restricted = new_df[new_df['codigo'].isin(['BOVA11','BRFS3','GOLL4','GFSA3','CMIN3','LEVE3','AMPB3','TRPL4','CVCB3','ASAI3','ROXO34'])].copy()

# new_df['str_corretora'] = new_df['str_corretora'].apply(lambda x: x.replace('Link','Guide'))
for x in new_df['str_corretora'].unique():
    output_file_path = f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Saldo-Dia-brokers\\{today.strftime('%d-%m-%Y')}\\Kapitalo_K11_{x}_{datetime.date.today().strftime('%Y-%m-%d')}.xlsx"
    aux_x = geral[geral['str_corretora']==x][['fundo','codigo','Doador']].copy()
    aux_x = aux_x[aux_x['Doador']>100]
    if os.path.exists(
        f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Saldo-Dia-brokers\\{today.strftime('%d-%m-%Y')}"
    ):

        aux_x.to_excel(output_file_path)
    else:
        os.mkdir(f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Saldo-Dia-brokers\\{today.strftime('%d-%m-%Y')}")
        aux_x.to_excel(output_file_path)
    
    # restricted[restricted['str_corretora']==x][['fundo','codigo','Doador']].to_excel(f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Saldo-Dia-brokers\\Kapitalo_K11__restricted{x}_{datetime.date.today().strftime('%Y-%m-%d')}.xlsx")

In [16]:
display(restricted.groupby(['fundo','codigo']).sum())
display(restricted.groupby(['codigo']).sum())

to_lend  saldo_dia_livre2  \
fundo                          codigo                              
KAPITALO K10 PREV MASTER FIM   LEVE3      6724             10524   
                               ROXO34   334272            422862   
KAPITALO KAPPA MASTER FIM      BOVA11   760838            900330   
                               LEVE3      1010              1010   
                               ROXO34  1027638           1296329   
KAPITALO KAPPA PREV MASTER FIM CMIN3      9464             11035   
                               ROXO34   122954            133064   
KAPITALO OMEGA PREV MASTER FIM CMIN3      1054              1323   
                               ROXO34    19971             25190   
KAPITALO SIGMA LLC             CMIN3       284              6958   
KAPITALO ZETA MASTER FIA       BOVA11   527298            527298   
                               BRFS3     20600            716696   
                               CMIN3      3688             13068   
                               GOLL4       370               882   
KAPITALO ZETA MASTER FIM       CMIN3    305032            517042   
                               LEVE3     10000             13100   

                                       dbl_quantidade  prop     Doador  
fundo                          codigo                                   
KAPITALO K10 PREV MASTER FIM   LEVE3         -99248.0   1.0     6724.0  
                               ROXO34        -42459.0   1.0   334272.0  
KAPITALO KAPPA MASTER FIM      BOVA11     -14185764.0   1.0   380419.0  
                               LEVE3         -99248.0   1.0     1010.0  
                               ROXO34        -42459.0   1.0  1027638.0  
KAPITALO KAPPA PREV MASTER FIM CMIN3        -723846.0   1.0     9464.0  
                               ROXO34        -42459.0   1.0   122954.0  
KAPITALO OMEGA PREV MASTER FIM CMIN3        -723846.0   1.0     1054.0  
                               ROXO34        -42459.0   1.0    19971.0  
KAPITALO SIGMA LLC             CMIN3       -1257241.0   1.0      142.0  
KAPITALO ZETA MASTER FIA       BOVA11     -14185764.0   1.0   263649.0  
                               BRFS3      -38205301.0   1.0    20600.0  
                               CMIN3       -1257241.0   1.0     1844.0  
                               GOLL4         -56283.0   1.0      370.0  
KAPITALO ZETA MASTER FIM       CMIN3       -1257241.0   1.0   152516.0  
                               LEVE3         -99248.0   1.0    10000.0

,to_lend,saldo_dia_livre2,dbl_quantidade,prop,Doador
codigo,,,,,
BOVA11,1288136,1427628,-28371528.0,2.0,644068.0
BRFS3,20600,716696,-38205301.0,1.0,20600.0
CMIN3,319522,549426,-5219415.0,5.0,165020.0
GOLL4,370,882,-56283.0,1.0,370.0
LEVE3,17734,24634,-297744.0,3.0,17734.0
ROXO34,1504835,1877445,-169836.0,4.0,1504835.0


In [41]:
bova = pd.read_excel(r'G:\Trading\K11\CashCarry\bova11\bova11_2023-11-06.xls',skiprows=1)[['Unnamed: 0','Unnamed: 1']]
bova.columns = ['Nome','Data']
bova.loc[bova['Nome']=='Data','Data'].item()
bova.loc[bova['Nome']=='Valor líquido dos ativos por quota**','Data'].item()

In [25]:
import imaplib
import email
from email import policy
from email.utils import parsedate_to_datetime
import datetime
import os

# Configuration
host = "imap.gmail.com"  # Replace with your IMAP server
LOGIN = os.getenv("EMAIL_USER")
# LOGIN = "joao.ramalho@kapitalo.com.br"

PASSWORD = os.getenv("EMAIL_PASSWORD")
search_keywords = ['ALUGUEL','DIA','KAPITALO'] 
directory_to_save = "G://Trading//K11//Aluguel//Trades//Necton//"  # Where to save attachments
filename_to_save_as = 'desired_filename'  # The name you want to give to the saved attachments
file_extensions = ['.xlsx']  # List of file extensions to be saved

# Connect to IMAP server
mail = imaplib.IMAP4_SSL(host)
mail.login(LOGIN, PASSWORD)
mail.select('inbox')  # Default is 'inbox'

def build_search_query(keywords):
    if not keywords:
        return 'ALL'
    if len(keywords) == 1:
        return f'(SUBJECT "{keywords[0]}" FROM "{keywords[0]}")'
    query = '(OR (SUBJECT "{}") (FROM "{}"))'.format(keywords[0], keywords[0])
    for keyword in keywords[1:]:
        query = f'(OR {query} (SUBJECT "{keyword}") (FROM "{keyword}"))'
    print(query)
    return query

# Search for emails
six_hours_ago = (datetime.datetime.now() - datetime.timedelta(hours=-6)).strftime('%d-%b-%Y')
search_query = build_search_query(search_keywords)
# subject = 'ALUGUEL DIA KAPITALO JOAO'
result, data = mail.uid('search', None, search_query, f'(SINCE {six_hours_ago})')


if result == 'OK':
    latest_time = None
    latest_attachment = None

    for num in data[0].split():
        result, data = mail.uid('fetch', num, '(RFC822)')
        if result == 'OK':
            email_message = email.message_from_bytes(data[0][1], policy=policy.default)
            # Check dates
            email_date = parsedate_to_datetime(email_message.get('Date'))
            if latest_time is None or email_date > latest_time:
                for part in email_message.walk():
                    if part.get_content_maintype() == 'multipart':
                        continue
                    if part.get('Content-Disposition') is None:
                        continue
                    file_name = part.get_filename()
                    if file_name:
                        _, file_extension = os.path.splitext(file_name)
                        if file_extension.lower() in file_extensions:
                            latest_time = email_date
                            latest_attachment = (num, part)
                            break

if latest_attachment:
    num, part = latest_attachment
    payload = part.get_payload(decode=True)
    file_name = f"{filename_to_save_as}{os.path.splitext(part.get_filename())[1]}"
    file_path = os.path.join(directory_to_save, file_name)
    with open(file_path, 'wb') as f:
        f.write(payload)
    print(f"Saved: {file_path}")
else:
    print("No attachment found in the last 6 hours with the given criteria.")

# Close the mail connection
mail.close()
mail.logout()

(OR (OR (OR (SUBJECT "ALUGUEL") (FROM "ALUGUEL")) (SUBJECT "DIA") (FROM "DIA")) (SUBJECT "KAPITALO") (FROM "KAPITALO"))
No attachment found in the last 6 hours with the given criteria.


('BYE', [b'LOGOUT Requested'])

In [24]:
result

'OK'

""


to_lend  saldo_dia_livre2  \
fundo                          codigo                              
KAPITALO K10 PREV MASTER FIM   BEEF3      6435              6435   
                               LEVE3      1234              6034   
KAPITALO KAPPA MASTER FIM      SANB11   583566            600966   
KAPITALO KAPPA PREV MASTER FIM BEEF3      6020              6020   
KAPITALO OMEGA PREV MASTER FIM BEEF3       391               785   
                               CMIN3      1054              1323   
                               SANB11      154               154   
KAPITALO SIGMA LLC             BEEF3        51                51   
                               CMIN3       284              6958   
                               SANB11       44               104   
KAPITALO ZETA MASTER FIA       BEEF3      3974              3974   
                               BOVA11  1119370           1119524   
                               BRFS3     20600            690295   
                               CMIN3      2502              7190   
                               GOLL4       882               882   
                               LWSA3      3714              4049   
                               MRFG3      5011            608952   
                               SANB11     6436             23222   
KAPITALO ZETA MASTER FIM       BEEF3     23934             23934   
                               SANB11     9414             44576   

                                       dbl_quantidade  prop    Doador  
fundo                          codigo                                  
KAPITALO K10 PREV MASTER FIM   BEEF3         -19124.0   1.0    6435.0  
                               LEVE3         -99248.0   1.0    1234.0  
KAPITALO KAPPA MASTER FIM      SANB11      -1237371.0   1.0  291783.0  
KAPITALO KAPPA PREV MASTER FIM BEEF3         -37424.0   1.0    3010.0  
KAPITALO OMEGA PREV MASTER FIM BEEF3         -19124.0   1.0     391.0  
                               CMIN3        -723846.0   1.0    1054.0  
                               SANB11       -410337.0   1.0     154.0  
KAPITALO SIGMA LLC             BEEF3       -1649865.0   1.0      51.0  
                               CMIN3       -1257241.0   1.0     142.0  
                               SANB11      -1237371.0   1.0      22.0  
KAPITALO ZETA MASTER FIA       BEEF3       -1649865.0   1.0    3974.0  
                               BOVA11     -13642685.0   1.0  559685.0  
                               BRFS3      -37325959.0   1.0   20600.0  
                               CMIN3       -1257241.0   1.0    1251.0  
                               GOLL4         -56283.0   1.0     882.0  
                               LWSA3       -3098393.0   1.0    3714.0  
                               MRFG3      -20229231.0   1.0    5011.0  
                               SANB11      -1237371.0   1.0    3218.0  
KAPITALO ZETA MASTER FIM       BEEF3       -1649865.0   1.0   23934.0  
                               SANB11      -1237371.0   1.0    4707.0

,to_lend,saldo_dia_livre2,dbl_quantidade,prop,Doador
codigo,,,,,
BEEF3,40805,41199,-5025267.0,6.0,37795.0
BOVA11,1119370,1119524,-13642685.0,1.0,559685.0
BRFS3,20600,690295,-37325959.0,1.0,20600.0
CMIN3,3840,15471,-3238328.0,3.0,2447.0
GOLL4,882,882,-56283.0,1.0,882.0
LEVE3,1234,6034,-99248.0,1.0,1234.0
LWSA3,3714,4049,-3098393.0,1.0,3714.0
MRFG3,5011,608952,-20229231.0,1.0,5011.0
SANB11,599614,669022,-5359821.0,5.0,299884.0
